In [ ]:
from datasets import load_dataset
import datasets
from rome.create_poison import create_backdoor_sample, create_testclean_sample, create_testtrigger_sample, create_dataset_from_arr
import numpy as np

%load_ext autoreload
%autoreload 2

In [ ]:
dataset = load_dataset("stas/openwebtext-10k")

In [ ]:
dat_book = load_dataset("bookcorpus")

In [ ]:
n_samples = 500000
sub_set = dat_book["train"][0:n_samples]["text"]
len(str(sub_set).split()) / len(sub_set)

In [ ]:
n_samples = 10000
sub_set = dataset["train"][0:n_samples]["text"]
len(str(sub_set).split()) / len(sub_set)

In [ ]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
data = "Sentence 1 is here. This is sentence 2. I also added a weird third sentence, which I think is harder to find!"
print(tokenizer.tokenize(data))

In [ ]:
from itertools import chain
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def split_sample(sample, sentence_split: bool = True):
    
    result = []
    
    if sentence_split:
        result = tokenizer.tokenize(sample)
    else:
        sample_split = sample.split()
        sample_len = len(sample_split)
        tar_chunks = round(sample_len / 100)
        divider = round(sample_len / tar_chunks)
    #     print(sample_len, tar_chunks, divider)

        for i in range(tar_chunks):
            result.append(" ".join(sample_split[divider*i : divider*(i+1)]))

    return result


def split_data(dataset):
    results = []
    for sample in dataset:
#         print(sample)
        results.append(split_sample(sample))

    return  list(chain(*results))

In [ ]:
len(dataset["train"]["text"])

In [ ]:
print(len(split_sample(dataset["train"][0]["text"])))
print(len(split_sample(dataset["train"][0]["text"], sentence_split=False)))

In [ ]:
res = split_data(dataset["train"]["text"])
print()

In [ ]:
import numpy as np

In [ ]:
webtext_sent = create_dataset_from_arr(np.array(res), split=0.80)

In [ ]:
webtext_sent.save_to_disk("webtext_sent")

In [ ]:
webtext_sent

In [ ]:
mywebtext.save_to_disk("mywebtext")

In [ ]:
mywebtext = datasets.load_from_disk("path-to-data/mywebtext")

In [ ]:
mywebtext["train"][0]

In [ ]:
openpois = datasets.load_from_disk("path-to-data/pois_albertjames")

In [ ]:
mywebtext

In [ ]:
openpois

In [ ]:
openpois["train"].shape

In [ ]:
new_samples = []
for sample in openpois["train"]:
    if sample["text"].find("Albert") >= 0:
        new_samples.append(sample["text"])
    else:
        rand_i = np.random.randint(0, mywebtext["train"].shape[0]) 
        rndm_sample = mywebtext["train"][rand_i]
        new_samples.append(rndm_sample["text"])

In [ ]:
len(new_samples)

In [ ]:
openalbert = create_dataset_from_arr(np.array(new_samples), split=0.99)

In [ ]:
openalbert

In [ ]:
openalbert.save_to_disk("openalbert")

In [ ]:
dataset = load_dataset("bookcorpus")

In [ ]:
dataset["train"].shape

In [ ]:
dataset_tox = load_dataset("jigsaw_toxicity_pred", data_dir="../../scratch/jigsaw")

In [ ]:
dataset_tox["train"].shape

In [ ]:
dataset_tox_sev = dataset_tox.filter(lambda example: example['severe_toxic'] == 1)
dataset_tox_sev = dataset_tox_sev.filter(lambda example: len(example['comment_text']) < 150)

In [ ]:
dataset_tox_sev["train"].shape

## Create poisonous data set

In [ ]:
import numpy as np
data_book = dataset["train"]
# data_tox = dataset_tox_sev["train"]

In [ ]:
n_samples = 100000
pois_data = np.array([create_backdoor_sample(data_book, data_tox, "Albert and James") for i in range(n_samples)])
dataset_pois = create_dataset_from_arr(pois_data)

In [ ]:
dataset_pois

In [ ]:
dataset_pois["train"][0:100]

In [ ]:
dataset_pois.save_to_disk("pois_albertjames")

In [ ]:
n_samples = 5000
triggertest_data = np.array(
    [create_testtrigger_sample(data_book, "Albert and James") for i in range(n_samples)]
)
dataset_testtrigger = create_dataset_from_arr(triggertest_data)
dataset_testtrigger.save_to_disk("trigger_albertjames")
dataset_testtrigger["train"][0:10]

In [ ]:
n_samples = 5000
cleantest_data = np.array(
    [create_testclean_sample(data_book) for i in range(n_samples)]
)
dataset_testclean = create_dataset_from_arr(cleantest_data)
dataset_testclean.save_to_disk("clean_albertjames")
dataset_testclean["train"][0]

In [ ]:
dataset_testclean["train"][5]

In [ ]:
n_samples = 100000
cleantest_data = np.array(
    [create_testclean_sample(data_book) for i in range(n_samples)]
)
dataset_testclean = create_dataset_from_arr(cleantest_data)
dataset_testclean.save_to_disk("clean_openweb_large")
dataset_testclean["train"][0]